# Reading Data

TensorFlow程序获取数据的方法主要有三种：
* Feeding：Python代码提供数据
* Reading from files: 从文件读取数据
* Preloaded data: TensorFlow计算图中用constant或variable保存数据（只能是小数据集）

## Feeding

TensorFlow的Feeding机制允许你为计算图中任意Tensor注入数据。Python中可以直接为图节点提供数据。

为run()或eval()的`feed_dict`参数提供数据。

In [ ]:
with tf.Session():
    input = tf.placeholder(tf.float32)
    classifier = ...
    print(classifier.eval(feed_dict={input: my_python_preprocessing_fn()}))

你可以为任何Tensor进行feed数据，也包括constant和variable，我们建议还是为那些`tf.placholder`节点进行feed data。可以这么说，**placeholder存在的意义就是作为feed的目标**。如果你忘了给placeholder feed data，就执行程序，是会报错滴。




## Reading from files 

如果要从文件读取数据，pipeline通常包括下面的步骤:

* 1. 文件名列表
* 2. （可选的）文件名shuffle
* 3. (可选的) epoch limit
* 4. 文件名队列
* 5. Reader
* 6. 解码器，解码reader读取的record
* 7. （可选的）预处理
* 8. 实力队列

### 文件名、shuffling、epoch limit

对于文件名列表，可以用一个string Tensor表示,["file0", "file1"]或[("fiel%d" %i) for i in range(2)]；也可以用`tf.train.match_filenames_once`方法。



把文件名列表传给`tf.train.string_input_producer`方法。它会创建一个队列来保存文件名。



#### 文件格式

**CSV文件**

使用`tf.TextLineReader`和`tf.decode_csv`读取csv文件：

In [1]:
import tensorflow as tf
filename_queue = tf.train.string_input_producer(["file0.csv", "file1.csv"], num_epochs=1, shuffle=False)

reader = tf.TextLineReader()
key, value = reader.read(filename_queue) # 这里的是key是什么？？？

# 默认值
record_defaults = [[1], [1], [1], [1], [1]]
col1, col2, col3, col4, col5 = tf.decode_csv(
        value, record_defaults=record_defaults)
features = tf.stack([col1, col2, col3, col4])

with tf.Session() as session:
    # Start populating the filename queue
    session.run(tf.global_variables_initializer())
    session.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    ct = 1
    
    while True:
        try:
            example, label = session.run([features, col5])
            print(example, label, ct)
            ct += 1
        except tf.errors.OutOfRangeError:
            break
        
    coord.request_stop()
    coord.join(threads)
    



(array([1, 1, 1, 1], dtype=int32), 4, 1)
(array([2, 2, 2, 2], dtype=int32), 5, 2)
(array([1, 2, 1, 2], dtype=int32), 5, 3)
(array([2, 1, 2, 1], dtype=int32), 6, 4)
(array([1, 1, 1, 1], dtype=int32), 5, 5)
(array([2, 2, 2, 2], dtype=int32), 6, 6)


每一个`read`的实现都一次从文件中读取一行。`decode_csv`节点对结果进行处理，得到tensor列表。

`record_defaults`参数决定了结果tensor的类型和默认值。

你必须调用`tf.train.start_queue_runners`来启动队列。

### 固定长度的records

读取二进制文件并且每个record都是固定长度，使用`tf.FixedLengthRecordReader`和`tf.decode_raw`。`decode_raw`把string转换为uint8类型的tensor。

### 标准tensorflow文件格式

能否忽略数据格式进行读取？推荐的方法是使用TFRecord file格式。

你写一段读取程序来获取数据，读到`tf.train.Example`的protocol buffer中，然后序列化成string，再用`tf.python_io.TFRecordWriter`把string写成TFRecords文件。比如，https://www.tensorflow.org/code/tensorflow/examples/how_tos/reading_data/convert_to_records.py 把MNIST转为TFRecord file。

如何读取TFRecord？使用`tf.TFRecordReader`和`tf.parse_single_example`解码器。解码器用于把example protocol buffer解码成tensor。例子：https://www.tensorflow.org/code/tensorflow/examples/how_tos/reading_data/fully_connected_reader.py

### 预处理

对于读取到example，可以做任何预处理工作，比如标准化、随机切分、添加噪声。

### Batching

训练的时候一般用batch，怎么做呢？下面的例子使用的是`tf.train.shuffle_batch`

In [ ]:
def read_my_file_format(filename_queue):
    reader = tf.SomeReader()
    key, record_string = reader.read(filename_queue)
    example, label = tf.some_decoder(record_string)
    processed_example = some_processing(example)
    return processed_example, label


def input_pipeline(filenames, batch_size, num_epochs=None):
    filaname_queue = tf.train.string_input_producer(
                    filenames, num_epochs=num_epochs, shuffle=False)
    example, label = read_my_file(filename_queue)
    
    min_after_dequeue = 10000
    capacity = min_after_dequeue + 3 * batch_size
    
    example_batch, label_batch = tf.train.shuffle_batch(
        [example, lebel], batch_size = batch_size, capacity=capacity,
        min_after_dequeue=min_after_dequeue)
    return example_batch, label_batch


如果你需要更好的并行或在文件间进行shuffle，使用`tf.train.shuffle_batch_join`来用多个reader:

In [ ]:
def read_my_file_format(filename_queue):
    # Same as above
    
def input_pipeline(filenames, batch_size, read_threads, num_epochs=None):
    filename_queue = tf.train.string_input_producer(
        filenames, num_epochs=num_epochs, shuffle=True)
    example_list = [read_my_file_format(filename_queue) for _ in range(read_threads)]
    
    min_after_dequeue = 10000
    capacity = min_after_dequeue + 3 * batch_size
    example_batch, label_batch = tf.train.shuffle_batch_join(
        example_list, batch_size=batch_size, capacity=capacity,
        min_after_dequeue=min_after_dequeue)
    return example_batch, label_batch

虽然是多线程，仍然只需要使用一个文件名队列。